# S&P 500 Earning Per Share Estimates
© 2025 Marek Ozana

Historical development of EPS estimates for S&P 500. Notice how estimates are always high for 3 years forward but then adjust to reality...

In [1]:
# Load hero chart from JSON file
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)
chart_file = Path("docs/examples/equity") / "spx-eps-estimates-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.LayerChart(...)

In [2]:
# Download data for multiple fiscal periods
import polars as pl

from polars_bloomberg import BQuery

with BQuery(timeout=50000) as con:
    res = []
    for n in range(1, 4):
        tbl = con.bql(f"""
            let(#eps=headline_eps_market(
                    fa_period_offset={n},
                    fa_period_type=A,
                    as_of_date=range('2009-12-31', TODAY(), frq=W)
                    );
            )
            get(#eps)
            for(['SPX Index'])
        """).combine()
        res.append(tbl)
data = pl.concat(res).drop_nulls().sort(by="AS_OF_DATE")
data.head(3)

ID,#eps,CURRENCY,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE
str,f64,str,date,date,date
"""SPX Index""",62.792159,"""USD""",2009-12-31,2009-12-31,2009-12-31
"""SPX Index""",79.130444,"""USD""",2009-12-31,2009-12-31,2010-12-31
"""SPX Index""",95.303385,"""USD""",2009-12-31,2009-12-31,2011-12-31


In [3]:
# Create chart
lines = (
    alt.Chart(data)
    .mark_line()
    .encode(
        x=alt.X("AS_OF_DATE:T").title(None),
        y=alt.Y("#eps:Q").scale(zero=False).title("EPS Estimate"),
        color=alt.Color("year(PERIOD_END_DATE):O")
        .scale(scheme="category20b")
        .title("Year"),
    )
)
# Plot text at the last point. !! Using argmax trick to get the last point. !!
last_point = lines.mark_point().encode(
    x=alt.X("max(AS_OF_DATE):T").title(None),
    y=alt.Y("#eps:Q").aggregate(argmax="AS_OF_DATE"),
)
last_bar = last_point.mark_rule(opacity=0.6).encode(y2=alt.datum(0))
text = last_point.mark_text(align="left", dx=3, opacity=0.6).encode(
    text=alt.Text("#eps:Q").aggregate(argmax="AS_OF_DATE").format("0.0f"),
)

chart = (lines + last_point + last_bar + text).properties(
    width=600, height=400, title="S&P EPS Estimates"
)
# Save the chart
chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=0.75)